In [1]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [2]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph, NodeMatcher
import pandas as pd

from rel2graph.relational_modules.pandas import PandasDataframeIterator
from rel2graph import IteratorIterator
from rel2graph import Converter

filename = "./pd_dataframe_demo_config.yaml"

In [3]:
# Configure Logging
import logging

#logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger("rel2graph")
logger.setLevel(logging.DEBUG)
log_formatter = logging.Formatter("%(asctime)s [%(threadName)s]::[%(levelname)s]::%(filename)s: %(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)


In [4]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
data = {
  "ID": [1,2,2,3,4,4],
  "FirstName": ["Julian", "Fritz",  "Fritz", "Hans", "Rudolfo", "Rudolfo"],
  "LastName": ["Minder", "Generic", "SomeGuy", "Müller", "Muster", "Muster"],
  "FavoriteFlower": ["virginica", "setosa", "setosa", "versicolor", "setosa", "setosa"]
}
people = pd.DataFrame(data)
people

,ID,FirstName,LastName,FavoriteFlower
0,1,Julian,Minder,virginica
1,2,Fritz,Generic,setosa
2,2,Fritz,SomeGuy,setosa
3,3,Hans,Müller,versicolor
4,4,Rudolfo,Muster,setosa
5,4,Rudolfo,Muster,setosa


In [6]:
graph = Graph(scheme="http", host="localhost", port=7474,  auth=('neo4j', 'password')) ## UPDATE password

graph.delete_all()  # reset graph (only wehn first creating the databse, here for debugging purposes)

In [7]:
iterator = IteratorIterator([PandasDataframeIterator(people, "Person"), PandasDataframeIterator(iris, "Flower")])

In [8]:
converter = Converter(filename, iterator, graph, num_workers=5)

2021-12-21 14:03:26,249 [MainThread]::[DEBUG]::config_parser.py: Parsing config file './pd_dataframe_demo_config.yaml'
2021-12-21 14:03:26,252 [MainThread]::[DEBUG]::config_parser.py: Compiling ENTITY("Flower") : {'NODE("Flower") flower': ['sepal_length = Flower.sepal_length', 'sepal_width = Flower.sepal_width', 'petal_length = Flower.petal_width', 'petal_width = Flower.petal_width'], 'NODE("Species", "BioEntity") species': ['+ Name = Flower.species'], 'RELATION(flower, "is", species)': None}
2021-12-21 14:03:26,256 [MainThread]::[DEBUG]::config_parser.py: Compiling module 'AttributeFactory' with static args ['sepal_length', 'sepal_length']
2021-12-21 14:03:26,257 [MainThread]::[DEBUG]::registrar.py: Requested factory 'AttributeFactory' from registry.
2021-12-21 14:03:26,257 [MainThread]::[DEBUG]::config_parser.py: Compiling module 'AttributeFactory' with static args ['sepal_width', 'sepal_width']
2021-12-21 14:03:26,257 [MainThread]::[DEBUG]::registrar.py: Requested factory 'Attribute

In [9]:
from tqdm.notebook import tqdm
converter(progress_bar=tqdm)

  0%|          | 0/312 [00:00<?, ?it/s]

2021-12-21 14:03:26,546 [MainThread]::[INFO]::converter.py: Running convertion with 5 parallel workers.
2021-12-21 14:03:26,547 [MainThread]::[INFO]::converter.py: Starting creation of nodes.
2021-12-21 14:03:26,548 [Worker-0]::[DEBUG]::converter.py: Starting Worker 0
2021-12-21 14:03:26,549 [Worker-1]::[DEBUG]::converter.py: Starting Worker 1
2021-12-21 14:03:26,549 [Worker-2]::[DEBUG]::converter.py: Starting Worker 2
2021-12-21 14:03:26,550 [Worker-3]::[DEBUG]::converter.py: Starting Worker 3
2021-12-21 14:03:26,550 [Worker-4]::[DEBUG]::converter.py: Starting Worker 4
2021-12-21 14:03:26,550 [Worker-0]::[DEBUG]::converter.py: Processing PandasDataframeResource 'Person' (row 0)
2021-12-21 14:03:26,551 [Worker-1]::[DEBUG]::converter.py: Processing PandasDataframeResource 'Person' (row 1)
2021-12-21 14:03:26,552 [Worker-2]::[DEBUG]::converter.py: Processing PandasDataframeResource 'Person' (row 2)
2021-12-21 14:03:26,553 [Worker-3]::[DEBUG]::converter.py: Processing PandasDataframeResou

In [11]:
converter._worker_pool.config.n_resources

156